Exploring Momentum Investing. I will be using high-quality momentum stocks by finding stocks which have carried momentum 
over a long period of time to avoid stocks which have high momentum for short periods of time.

In [2]:
import numpy as np 
import pandas as pd
import requests 
import math 
from scipy import stats
import xlsxwriter 

ModuleNotFoundError: No module named 'xlsxwriter'

For this bot, I will be using stocks that are within the S&P 500.

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

API Calls

In [10]:
symbol = 'AMD'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


{'symbol': 'AMD',
 'companyName': 'Advanced Micro Devices Inc.',
 'primaryExchange': 'TQEK/ASAGLAO B)RMLCSSL NAGED E(NT',
 'calculationPrice': 'close',
 'open': 92.74,
 'openTime': 1663760145941,
 'openSource': 'cafliiof',
 'close': 93.22,
 'closeTime': 1684416710670,
 'closeSource': 'lifcaofi',
 'high': 94.63,
 'highTime': 1670882227360,
 'highSource': 'iideeer ncad pyeu1l mt5',
 'low': 90.97,
 'lowTime': 1623333700721,
 'lowSource': 'i5 p dyelcirem1etunea d',
 'latestPrice': 92.21,
 'latestSource': 'Close',
 'latestTime': 'December 10, 2020',
 'latestUpdate': 1679351694099,
 'latestVolume': 35097619,
 'iexRealtimePrice': 93.14,
 'iexRealtimeSize': 1,
 'iexLastUpdated': 1646344526013,
 'delayedPrice': 95.46,
 'delayedPriceTime': 1666179640644,
 'oddLotDelayedPrice': 93.31,
 'oddLotDelayedPriceTime': 1609402890014,
 'extendedPrice': 95.5,
 'extendedChange': -0.16,
 'extendedChangePercent': -0.00179,
 'extendedPriceTime': 1664437296756,
 'previousClose': 93.17,
 'previousVolume': 547095

In [11]:
data['ytdChange']

1.0431655766880745

Split S&P 500 into groups of 100 each (with an extra group of remaining stocks). 

In [102]:
def splitEven(stocks, batch_size):
    for i in range(0, len(stocks), batch_size):
        yield stocks[i: i+ batch_size]
symbol_groups = list(splitEven(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
cols = [
                'Ticker', 
                'Price', 
                'Shares to Purchase', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'Final Score'
                ]
time_intervals = ['One-Month', 'Three-Month', 'Six-Month', 'One-Year']

In [103]:
#grab data for each stock including momentum on different timelines
df = pd.DataFrame(columns = cols)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series([symbol, 
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],   
                'N/A',
                'N/A'
                ], 
            index = cols), 
            ignore_index = True)
df
    

,Ticker,Price,Shares to Purchase,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Final Score
0,A,119.92,N/A,0.460405,N/A,0.343819,N/A,0.21259,N/A,0.078758,N/A,N/A
1,AAL,18.07,N/A,-0.343922,N/A,0.0581757,N/A,0.392346,N/A,0.454951,N/A,N/A
2,AAP,161.67,N/A,0.0201859,N/A,0.131901,N/A,0.0323237,N/A,0.0181229,N/A,N/A
3,AAPL,125.33,N/A,0.884807,N/A,0.402588,N/A,0.0898836,N/A,0.0648728,N/A,N/A
4,ABBV,109.57,N/A,0.322261,N/A,0.142417,N/A,0.224611,N/A,0.0911244,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.69,N/A,0.0959629,N/A,0.143998,N/A,0.160126,N/A,0.0391593,N/A,N/A
501,ZBH,152.34,N/A,0.0133707,N/A,0.0940793,N/A,0.0564081,N/A,-0.0287032,N/A,N/A
502,ZBRA,375.40,N/A,0.478965,N/A,0.42085,N/A,0.506444,N/A,0.0905182,N/A,N/A
503,ZION,42.81,N/A,-0.114427,N/A,0.184961,N/A,0.397302,N/A,0.0970916,N/A,N/A


In [106]:
df = df.dropna()
#print(df)
for row in df.index:
    #print(row)
    for interval in time_intervals:
        df.loc[row, f'{interval} Return Percentile']  = stats.percentileofscore(df[f'{interval} Price Return'], 
                                  df.loc[row, f'{interval} Price Return'])/100
#loop through and show percentiles
for interval in time_intervals:
    print(df[f'{interval} Return Percentile'])

#Print the entire DataFrame    
#df

0        0.674651
1         0.99002
2        0.411178
3        0.630739
4        0.710579
5       0.0299401
6        0.285429
7        0.129741
8        0.483034
9         0.59481
10       0.580838
11       0.237525
12       0.371257
13       0.826347
14      0.0758483
15      0.0319361
16       0.321357
17       0.724551
18       0.508982
19       0.762475
20       0.267465
21       0.592814
22       0.654691
23       0.902196
24       0.728543
25       0.812375
26       0.798403
27       0.123752
28      0.0658683
29       0.954092
30       0.207585
31        0.88024
32       0.413174
33      0.0998004
34       0.582834
35      0.0858283
36       0.433134
37       0.664671
38       0.550898
39       0.115768
40        0.47505
41       0.467066
42       0.996008
43     0.00598802
44       0.676647
45       0.836327
46        0.60479
47       0.135729
48       0.760479
49      0.0838323
50       0.816367
51       0.343313
52      0.0279441
53        0.56487
54       0.163673
55       0

Now that we have the momentum stocks and their percentiles, we will calculate their overall score using the percentiles.

In [112]:
from statistics import mean

for row in df.index:
    percentiles = []
    for interval in time_intervals:
        percentiles.append(df.loc[row, f'{interval} Return Percentile'])
    df.loc[row, 'Final Score'] = mean(percentiles)
df

,Ticker,Price,Shares to Purchase,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Final Score,Score
0,A,119.92,N/A,0.460405,0.89022,0.343819,0.810379,0.21259,0.628743,0.078758,0.674651,0.750998,0.750998
1,AAL,18.07,N/A,-0.343922,0.0399202,0.0581757,0.265469,0.392346,0.886228,0.454951,0.99002,0.545409,0.545409
2,AAP,161.67,N/A,0.0201859,0.37525,0.131901,0.423154,0.0323237,0.221557,0.0181229,0.411178,0.357784,0.357784
3,AAPL,125.33,N/A,0.884807,0.98004,0.402588,0.89022,0.0898836,0.371257,0.0648728,0.630739,0.718064,0.718064
4,ABBV,109.57,N/A,0.322261,0.794411,0.142417,0.441118,0.224611,0.654691,0.0911244,0.710579,0.6502,0.650200
5,ABC,102.31,N/A,0.168991,0.616766,0.0342596,0.221557,0.0608521,0.307385,-0.0941433,0.0299401,0.293912,0.293912
6,ABMD,279.87,N/A,0.506236,0.904192,0.0702279,0.279441,-0.0156987,0.115768,-0.00350654,0.285429,0.396208,0.396208
7,ABT,106.80,N/A,0.285251,0.762475,0.167495,0.50499,0.0285804,0.217565,-0.0393349,0.129741,0.403693,0.403693
8,ACN,248.87,N/A,0.236026,0.688623,0.188096,0.560878,0.0588877,0.301397,0.0324539,0.483034,0.508483,0.508483
9,ADBE,494.49,N/A,0.587239,0.932136,0.178796,0.540918,0.00133973,0.147705,0.0569726,0.59481,0.553892,0.553892


Remove everything but the top 100 scored stocks

In [118]:
df.sort_values(by = 'Final Score', ascending = False)

,Ticker,Price,Shares to Purchase,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Final Score,Score
23,ALB,141.68,N/A,1.23561,0.992016,0.707681,0.978044,0.536467,0.956088,0.210194,0.902196,0.957086,0.957086
29,AMAT,89.40,N/A,0.576102,0.93014,0.478507,0.918164,0.599269,0.968064,0.281835,0.954092,0.942615,0.942615
24,ALGN,519.92,N/A,0.871516,0.978044,0.912464,0.99002,0.571578,0.964072,0.0953118,0.728543,0.91517,0.915170
45,APTV,130.16,N/A,0.354362,0.826347,0.520708,0.948104,0.477551,0.944112,0.1434,0.836327,0.888723,0.888723
31,AMD,93.41,N/A,1.34706,0.994012,0.604291,0.962076,0.166577,0.526946,0.177769,0.88024,0.840818,0.840818
66,BKR,23.99,N/A,0.0813203,0.479042,0.372636,0.854291,0.707501,0.986028,0.323563,0.97006,0.822355,0.822355
82,CBRE,67.77,N/A,0.134786,0.578842,0.292856,0.754491,0.40076,0.898204,0.174209,0.878244,0.777445,0.777445
67,BLK,704.24,N/A,0.459666,0.888224,0.27318,0.726547,0.302836,0.790419,0.0656496,0.632735,0.759481,0.759481
78,CARR,38.59,N/A,2.12847,1,0.724661,0.98004,0.293976,0.776447,-0.00722384,0.269461,0.756487,0.756487
13,ADSK,291.41,N/A,0.592433,0.936128,0.19035,0.566866,0.236211,0.682635,0.138406,0.826347,0.752994,0.752994
